<a id='top-page'></a>

# Visualizing the results of running the [`AE-transport.py`](../code/AE-transport.py) script

- [Upload the results](#load-results)
- [Load the training data](#load-data)
- [Compute the low-dimensional manifold parameters](#PCA)
- [Compute the *a priori* closure model assessments](#closure-model-assessment)
- [Visualize the numerical solution](#visualize-numerical-solution)
- [Visualize the transported trajectories](#visualize-trajectories)
- [Visualize the *a posteriori* predictions of the projected source terms](#visualize-source-terms)
- [Visualize the predicted thermo-chemical state variables](#visualize-thermo-chemistry)
- [Assess the thermo-chemistry prediction](#assess-thermo-chemistry-prediction)

***

In [ ]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

from PCAfold import preprocess
from PCAfold import reduction
from PCAfold import analysis
from PCAfold.styles import *

%matplotlib inline

In [ ]:
color_rbf = '#f09863'
color_gpr = '#ff5555'
color_ann = '#1c8aa9'
color_kreg = '#0e3a61'

data_tag = 'CO-H2-10-1-isobaric-adiabatic-closed-HR'
data_path = '../data/'
results_path = '../results/'
species_to_remove_list = ['N2', 'AR', 'HE']
sample_percentage = 100
random_seed = 100
n_components = 2
scaling = '0to1'
run_RBF = True
run_GPR = False
run_ANN = True
run_KReg = True
reconstruct_thermochemistry = True
start_simulation = 120
n_points = 2000
max_simulation_time = 0.005

<a id='load-results'></a>
***

## Upload the results

[Go to the top ↑](#top-page)

In [ ]:
if run_RBF:
    AE_source_terms_predicted_rbf = pd.read_csv(results_path + 'best-AE-RBF-predicted-source-terms.csv', sep = ',', header=None).to_numpy()
    numerical_solution_rbf_model = pd.read_csv(results_path + 'best-AE-RBF-numerical-solution.csv', sep = ',', header=None).to_numpy()
    AE_source_terms_from_numerical_solution_rbf = pd.read_csv(results_path + 'best-AE-RBF-AE-source-terms-from-numerical-solution.csv', sep = ',', header=None).to_numpy()

if run_GPR:
    AE_source_terms_predicted_gpr = pd.read_csv(results_path + 'best-AE-GPR-predicted-source-terms.csv', sep = ',', header=None).to_numpy()
    numerical_solution_gpr_model = pd.read_csv(results_path + 'best-AE-GPR-numerical-solution.csv', sep = ',', header=None).to_numpy()
    AE_source_terms_from_numerical_solution_gpr = pd.read_csv(results_path + 'best-AE-GPR-AE-source-terms-from-numerical-solution.csv', sep = ',', header=None).to_numpy()

if run_ANN:
    AE_source_terms_predicted_ann = pd.read_csv(results_path + 'best-AE-ANN-predicted-source-terms.csv', sep = ',', header=None).to_numpy()
    numerical_solution_ann_model = pd.read_csv(results_path + 'best-AE-ANN-numerical-solution.csv', sep = ',', header=None).to_numpy()
    AE_source_terms_from_numerical_solution_ann = pd.read_csv(results_path + 'best-AE-ANN-AE-source-terms-from-numerical-solution.csv', sep = ',', header=None).to_numpy()

if run_KReg:
    AE_source_terms_predicted_kreg = pd.read_csv(results_path + 'best-AE-KReg-predicted-source-terms.csv', sep = ',', header=None).to_numpy()
    numerical_solution_kreg_model = pd.read_csv(results_path + 'best-AE-KReg-numerical-solution.csv', sep = ',', header=None).to_numpy()
    AE_source_terms_from_numerical_solution_kreg = pd.read_csv(results_path + 'best-AE-KReg-AE-source-terms-from-numerical-solution.csv', sep = ',', header=None).to_numpy()

if reconstruct_thermochemistry:
    
    if run_RBF: state_space_ROM_predicted_rbf = pd.read_csv(results_path + 'best-AE-RBF-predicted-state-space.csv', sep = ',', header=None).to_numpy()
    if run_GPR: state_space_ROM_predicted_gpr = pd.read_csv(results_path + 'best-AE-GPR-predicted-state-space.csv', sep = ',', header=None).to_numpy()
    if run_ANN: state_space_ROM_predicted_ann = pd.read_csv(results_path + 'best-AE-ANN-predicted-state-space.csv', sep = ',', header=None).to_numpy()
    if run_KReg: state_space_ROM_predicted_kreg = pd.read_csv(results_path + 'best-AE-KReg-predicted-state-space.csv', sep = ',', header=None).to_numpy()

<a id='load-data'></a>
***

## Load the training data

[Go to the top ↑](#top-page)

In [ ]:
state_space_names = pd.read_csv(data_path + data_tag + '-state-space-names.csv', sep = ',', header=None).to_numpy().ravel()
state_space = pd.read_csv(data_path + data_tag + '-state-space.csv', sep = ',', header=None).to_numpy()
state_space_sources = pd.read_csv(data_path + data_tag + '-state-space-sources.csv', sep = ',', header=None).to_numpy()
mixture_fraction = pd.read_csv(data_path + data_tag + '-mixture-fraction.csv', sep = ',', header=None).to_numpy()
time_grid = pd.read_csv(data_path + data_tag + '-time.csv', sep = ',', header=None).to_numpy().ravel()

state_space_test_trajectory = pd.read_csv(data_path + data_tag + '-state-space-test-trajectory.csv', sep = ',', header=None).to_numpy()
state_space_sources_test_trajectory = pd.read_csv(data_path + data_tag + '-state-space-sources-test-trajectory.csv', sep = ',', header=None).to_numpy()
time_grid_test_trajectory = pd.read_csv(data_path + data_tag + '-time-test-trajectory.csv', sep = ',', header=None).to_numpy().ravel()

state_space_support_trajectories = pd.read_csv(data_path + data_tag + '-state-space-support-trajectories.csv', sep = ',', header=None).to_numpy()
state_space_sources_support_trajectories = pd.read_csv(data_path + data_tag + '-state-space-sources-support-trajectories.csv', sep = ',', header=None).to_numpy()
time_grid_support_trajectories = pd.read_csv(data_path + data_tag + '-time-support-trajectories.csv', sep = ',', header=None).to_numpy().ravel()

for species_to_remove in species_to_remove_list:

    (species_index, ) = np.where(state_space_names==species_to_remove)
    if len(species_index) != 0:
        print('Removing ' + state_space_names[int(species_index)] + '.')
        state_space = np.delete(state_space, np.s_[species_index], axis=1)
        state_space_sources = np.delete(state_space_sources, np.s_[species_index], axis=1)

        state_space_test_trajectory = np.delete(state_space_test_trajectory, np.s_[species_index], axis=1)
        state_space_sources_test_trajectory = np.delete(state_space_sources_test_trajectory, np.s_[species_index], axis=1)

        state_space_support_trajectories = np.delete(state_space_support_trajectories, np.s_[species_index], axis=1)
        state_space_sources_support_trajectories = np.delete(state_space_sources_support_trajectories, np.s_[species_index], axis=1)

        state_space_names = np.delete(state_space_names, np.s_[species_index])
    else:
        print(species_to_remove + ' already removed from the data set.')

(n_observations, n_variables) = np.shape(state_space)

if sample_percentage == 100:
    sample_data = False
else:
    sample_data = True

if sample_data:
    idx = np.zeros((n_observations,)).astype(int)
    sample_random = preprocess.DataSampler(idx, random_seed=random_seed, verbose=False)
    (idx_sample, _) = sample_random.random(sample_percentage)

    state_space = state_space[idx_sample,:]
    state_space_sources = state_space_sources[idx_sample,:]
    mf = mf[idx_sample,:]

    (n_observations, n_variables) = np.shape(state_space)

print('\nThe data set has ' + str(n_observations) + ' observations.')

<a id='PCA'></a>
***

## Compute the low-dimensional manifold parameters

[Go to the top ↑](#top-page)

In [ ]:
AE_basis = np.array([[ 0.44541207, -0.53351498],
       [ 0.21240635,  0.03709494],
       [ 0.23040931,  0.0683625 ],
       [ 0.04429741,  0.27895769],
       [ 0.17797215,  0.02717828],
       [-0.77823412, -0.22460838],
       [ 0.01838836,  0.13045858],
       [-0.06177867, -0.02877426],
       [-0.22748888,  0.42358291],
       [ 0.02015193, -0.61245435],
       [-0.08835387, -0.08909094]])

(X_CS, centers, scales) = preprocess.center_scale(state_space, scaling=scaling)
S_CS = state_space_sources / scales

state_space_test_trajectory_CS = (state_space_test_trajectory - centers)/scales
state_space_sources_test_trajectory_CS = state_space_sources_test_trajectory / scales

state_space_support_trajectories_CS = (state_space_support_trajectories - centers)/scales
state_space_sources_support_trajectories_CS = state_space_sources_support_trajectories / scales

X_AE = np.dot(X_CS, AE_basis)
S_AE = np.dot(S_CS, AE_basis)

X_AE_CS, center_AE, scale_AE = preprocess.center_scale(X_AE, scaling='-1to1')

AE_state_space_test_trajectory = np.dot(state_space_test_trajectory_CS, AE_basis)
AE_state_space_test_trajectory_CS = (AE_state_space_test_trajectory - center_AE) / scale_AE
AE_source_terms_test_trajectory = np.dot(state_space_sources_test_trajectory_CS, AE_basis)

AE_state_space_support_trajectories = np.dot(state_space_support_trajectories_CS, AE_basis)
AE_state_space_support_trajectories_CS = (AE_state_space_support_trajectories - center_AE) / scale_AE
AE_source_terms_support_trajectories = np.dot(state_space_sources_support_trajectories_CS, AE_basis)

AE_source_terms_support_trajectories_CS, centers_AE_source_terms, scales_AE_source_terms = preprocess.center_scale(AE_source_terms_support_trajectories, scaling='-1to1')

(n_observations, _) = AE_state_space_support_trajectories.shape

In [ ]:
AE_initial_condition = AE_state_space_test_trajectory[start_simulation,:]
t_coordinates = np.linspace(time_grid_test_trajectory[start_simulation],max_simulation_time,n_points)

<a id='closure-model-assessment'></a>
***

## Compute the *a priori* closure model assessments

[Go to the top ↑](#top-page)

In [ ]:
if run_RBF:
    RBF_metrics = analysis.RegressionAssessment(AE_source_terms_test_trajectory, AE_source_terms_predicted_rbf, variable_names=['$S_{\eta, 1}$', '$S_{\eta, 2}$'])
    RBF_metrics.print_metrics(table_format=['pandas'], float_format='.8f', metrics=['R2', 'MSLE', 'NRMSE', 'GDE'])

In [ ]:
if run_GPR:
    GPR_metrics = analysis.RegressionAssessment(AE_source_terms_test_trajectory, AE_source_terms_predicted_gpr, variable_names=['$S_{\eta, 1}$', '$S_{\eta, 2}$'])
    GPR_metrics.print_metrics(table_format=['pandas'], float_format='.8f', metrics=['R2', 'MSLE', 'NRMSE', 'GDE'])

In [ ]:
if run_ANN:
    ANN_metrics = analysis.RegressionAssessment(AE_source_terms_test_trajectory, AE_source_terms_predicted_ann, variable_names=['$S_{\eta, 1}$', '$S_{\eta, 2}$'])
    ANN_metrics.print_metrics(table_format=['pandas'], float_format='.8f', metrics=['R2', 'MSLE', 'NRMSE', 'GDE'])

In [ ]:
if run_KReg:
    KReg_metrics = analysis.RegressionAssessment(AE_source_terms_test_trajectory, AE_source_terms_predicted_kreg, variable_names=['$S_{\eta, 1}$', '$S_{\eta, 2}$'])
    KReg_metrics.print_metrics(table_format=['pandas'], float_format='.8f', metrics=['R2', 'MSLE', 'NRMSE', 'GDE'])

<a id='visualize-numerical-solution'></a>
***

## Visualize the numerical solution

[Go to the top ↑](#top-page)

In [ ]:
plt.figure(figsize=(18,4))

plt.semilogx(time_grid_test_trajectory, AE_state_space_test_trajectory[:,0], 'k--', lw=5, zorder=5)
plt.xlim([10**-4,10**-2])

if run_RBF: plt.semilogx(t_coordinates, numerical_solution_rbf_model[:,0], '-', lw=2, color=color_rbf, zorder=40)
if run_GPR: plt.semilogx(t_coordinates, numerical_solution_gpr_model[:,0], '-', lw=3, color=color_gpr, zorder=30)
if run_ANN: plt.semilogx(t_coordinates, numerical_solution_ann_model[:,0], '-', lw=4, color=color_ann, zorder=20)
if run_KReg: plt.semilogx(t_coordinates, numerical_solution_kreg_model[:,0], '-', lw=5, color=color_kreg, zorder=10)

plt.xlabel('Time, [$s$]', fontsize=20)
plt.ylabel('$Z_{1}$', fontsize=20, rotation=0, horizontalalignment='right')
plt.grid(alpha=0.3, zorder=1);
plt.savefig(results_path + 'best-AE-a-posteriori-Z1-path.png', dpi=300, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(18,4))

plt.semilogx(time_grid_test_trajectory, AE_state_space_test_trajectory[:,1], 'k--', lw=5, zorder=5)
plt.xlim([10**-4,10**-2])

if run_RBF: plt.semilogx(t_coordinates, numerical_solution_rbf_model[:,1], '-', lw=2, color=color_rbf, zorder=40)
if run_GPR: plt.semilogx(t_coordinates, numerical_solution_gpr_model[:,1], '-', lw=3, color=color_gpr, zorder=30)
if run_ANN: plt.semilogx(t_coordinates, numerical_solution_ann_model[:,1], '-', lw=4, color=color_ann, zorder=20)
if run_KReg: plt.semilogx(t_coordinates, numerical_solution_kreg_model[:,1], '-', lw=5, color=color_kreg, zorder=10)

plt.xlabel('Time, [$s$]', fontsize=20)
plt.ylabel('$Z_{2}$', fontsize=20, rotation=0, horizontalalignment='right')
plt.grid(alpha=0.3, zorder=1)
plt.savefig(results_path + 'best-AE-a-posteriori-Z2-path.png', dpi=300, bbox_inches='tight')

<a id='visualize-trajectories'></a>
***

## Visualize the transported trajectories

[Go to the top ↑](#top-page)

In [ ]:
if run_RBF:

    numerical_solution_rbf_model_CS = (numerical_solution_rbf_model - center_AE) / scale_AE

    plt = reduction.plot_2d_manifold(X_AE_CS[:,0],
                                     X_AE_CS[:,1],
                                     s=20,
                                     color=S_AE[:,0],
                                     color_map='inferno',
                                     x_label='$Z_1$',
                                     y_label='$Z_2$',
                                     colorbar_label='$S_{Z, 1}$',
                                     figure_size=(18,5));

    plt.scatter(numerical_solution_rbf_model_CS[:,0], numerical_solution_rbf_model_CS[:,1], c=color_rbf, s=30, zorder=100)
    plt.scatter(AE_state_space_test_trajectory_CS[:,0], AE_state_space_test_trajectory_CS[:,1], c='k', s=10, zorder=1);
    plt.savefig(results_path + 'best-AE-RBF-transported-trajectory.png', dpi=300, bbox_inches='tight')

In [ ]:
if run_GPR:

    numerical_solution_gpr_model_CS = (numerical_solution_gpr_model - center_AE) / scale_AE

    plt = reduction.plot_2d_manifold(X_AE_CS[:,0],
                                     X_AE_CS[:,1],
                                     s=20,
                                     color=S_AE[:,0],
                                     color_map='inferno',
                                     x_label='$Z_1$',
                                     y_label='$Z_2$',
                                     colorbar_label='$S_{Z, 1}$',
                                     figure_size=(18,5));

    plt.scatter(numerical_solution_gpr_model_CS[:,0], numerical_solution_gpr_model_CS[:,1], c=color_gpr, s=30, zorder=100)
    plt.scatter(AE_state_space_test_trajectory_CS[:,0], AE_state_space_test_trajectory_CS[:,1], c='k', s=10, zorder=1);
    plt.savefig(results_path + 'best-AE-GPR-transported-trajectory.png', dpi=300, bbox_inches='tight')

In [ ]:
if run_ANN:
    
    numerical_solution_ann_model_CS = (numerical_solution_ann_model - center_AE) / scale_AE

    plt = reduction.plot_2d_manifold(X_AE_CS[:,0],
                                     X_AE_CS[:,1],
                                     s=20,
                                     color=S_AE[:,0],
                                     color_map='inferno',
                                     x_label='$Z_1$',
                                     y_label='$Z_2$',
                                     colorbar_label='$S_{Z, 1}$',
                                     figure_size=(18,5));

    plt.scatter(numerical_solution_ann_model_CS[:,0], numerical_solution_ann_model_CS[:,1], c=color_ann, s=30, zorder=100)
    plt.scatter(AE_state_space_test_trajectory_CS[:,0], AE_state_space_test_trajectory_CS[:,1], c='k', s=10, zorder=1);
    plt.savefig(results_path + 'best-AE-ANN-transported-trajectory.png', dpi=300, bbox_inches='tight')

In [ ]:
if run_KReg:
    
    numerical_solution_kreg_model_CS = (numerical_solution_kreg_model - center_AE) / scale_AE

    plt = reduction.plot_2d_manifold(X_AE_CS[:,0],
                                     X_AE_CS[:,1],
                                     s=20,
                                     color=S_AE[:,0],
                                     color_map='inferno',
                                     x_label='$Z_1$',
                                     y_label='$Z_2$',
                                     colorbar_label='$S_{Z, 1}$',
                                     figure_size=(18,5));

    plt.scatter(numerical_solution_kreg_model_CS[:,0], numerical_solution_kreg_model_CS[:,1], c=color_kreg, s=40, zorder=100)
    plt.scatter(AE_state_space_test_trajectory_CS[:,0], AE_state_space_test_trajectory_CS[:,1], c='k', s=10, zorder=1);
    plt.savefig(results_path + 'best-AE-KReg-transported-trajectory.png', dpi=300, bbox_inches='tight')

<a id='visualize-source-terms'></a>

***

## Visualize the *a posteriori* predictions of the projected source terms

[Go to the top ↑](#top-page)

In [ ]:
plt.figure(figsize=(18,4))

plt.semilogx(time_grid_test_trajectory, AE_source_terms_test_trajectory[:,0], 'k--', lw=5, zorder=5)
plt.xlim([10**-4,10**-2])

if run_RBF: plt.semilogx(t_coordinates, AE_source_terms_from_numerical_solution_rbf[:,0], '-', lw=2, color=color_rbf, zorder=40)
if run_GPR: plt.semilogx(t_coordinates, AE_source_terms_from_numerical_solution_gpr[:,0], '-', lw=3, color=color_gpr, zorder=30)
if run_ANN: plt.semilogx(t_coordinates, AE_source_terms_from_numerical_solution_ann[:,0], '-', lw=4, color=color_ann, zorder=20)
if run_KReg: plt.semilogx(t_coordinates, AE_source_terms_from_numerical_solution_kreg[:,0], '-', lw=5, color=color_kreg, zorder=10)

plt.xlabel('Time, [$s$]', fontsize=20)
plt.ylabel('$S_{Z, 1}$', fontsize=20, rotation=0, horizontalalignment='right')
plt.grid(alpha=0.3, zorder=1)
plt.savefig(results_path + 'best-AE-a-posteriori-SZ1-prediction.png', dpi=300, bbox_inches='tight');

In [ ]:
plt.figure(figsize=(18,4))

plt.semilogx(time_grid_test_trajectory, AE_source_terms_test_trajectory[:,1], 'k--', lw=5, zorder=5)
plt.xlim([10**-4,10**-2])

if run_RBF: plt.semilogx(t_coordinates, AE_source_terms_from_numerical_solution_rbf[:,1], '-', lw=2, color=color_rbf, zorder=40)
if run_GPR: plt.semilogx(t_coordinates, AE_source_terms_from_numerical_solution_gpr[:,1], '-', lw=3, color=color_gpr, zorder=30)
if run_ANN: plt.semilogx(t_coordinates, AE_source_terms_from_numerical_solution_ann[:,1], '-', lw=4, color=color_ann, zorder=20)
if run_KReg: plt.semilogx(t_coordinates, AE_source_terms_from_numerical_solution_kreg[:,1], '-', lw=5, color=color_kreg, zorder=10)

plt.xlabel('Time, [$s$]', fontsize=20)
plt.ylabel('$S_{Z, 2}$', fontsize=20, rotation=0, horizontalalignment='right')
plt.grid(alpha=0.3, zorder=1)
plt.savefig(results_path + 'best-AE-a-posteriori-SZ2-prediction.png', dpi=300, bbox_inches='tight');

<a id='visualize-thermo-chemistry'></a>

***

## Visualize the predicted thermo-chemical state variables

[Go to the top ↑](#top-page)

In [ ]:
if reconstruct_thermochemistry:
    
    selected_state_variables = [0,1,2,3,4,5,6,7,8,9,10]
    state_space_names_neat = ['$T$ [$K$]', '$Y_{H_2}$ [$-$]', '$Y_{O_2}$ [$-$]', '$Y_{O}$ [$-$]', '$Y_{OH}$ [$-$]', '$Y_{H_2O}$ [$-$]', '$Y_{H}$ [$-$]', '$Y_{HO_2}$ [$-$]', '$Y_{CO}$ [$-$]', '$Y_{CO_2}$ [$-$]', '$Y_{HCO}$ [$-$]']

    figure = plt.figure(figsize=(25, 16))
    spec = figure.add_gridspec(ncols=3, nrows=int(np.ceil(n_variables/3)), width_ratios=[1,1,1], height_ratios=[1 for i in range(0,int(np.ceil(n_variables/3)))])
    for i in range(0,len(selected_state_variables)):
        figure_a = figure.add_subplot(spec[int(np.floor(i/3)),i%3])
        plt.semilogx(time_grid_test_trajectory, state_space_test_trajectory[:,selected_state_variables[i]], 'k--', lw=4, alpha=1, zorder=5, label='True test trajectory')

        if run_RBF: plt.semilogx(t_coordinates, state_space_ROM_predicted_rbf[:,i], c=color_rbf, lw=2, zorder=40, label='RBF closure model')
        if run_GPR: plt.semilogx(t_coordinates, state_space_ROM_predicted_gpr[:,i], c=color_gpr, lw=3, zorder=30, label='GPR closure model')
        if run_ANN: plt.semilogx(t_coordinates, state_space_ROM_predicted_ann[:,i], c=color_ann, lw=4, zorder=20, label='ANN closure model')
        if run_KReg: plt.semilogx(t_coordinates, state_space_ROM_predicted_kreg[:,i], c=color_kreg, lw=5, zorder=10, label='Kernel regression closure model')
        plt.xlim([10**-4,10**-2])
        plt.grid(alpha=0.3, zorder=1);
        plt.ylabel(state_space_names_neat[selected_state_variables[i]], fontsize=20, rotation=90)
        if i==len(selected_state_variables)-3: plt.xlabel('$t$ [$s$]', fontsize=20)
        if i==len(selected_state_variables)-2: plt.xlabel('$t$ [$s$]', fontsize=20)
        if i==len(selected_state_variables)-1: plt.xlabel('$t$ [$s$]', fontsize=20)
        plt.gca().spines["top"].set_visible(False)
        plt.gca().spines["right"].set_visible(False)

        if i==len(selected_state_variables)-1: plt.legend(frameon=False, bbox_to_anchor=(2.2, 0.7), fontsize=20, ncol=1)

    plt.savefig(results_path + 'best-AE-predicted-thermo-chemistry.png', dpi=300, bbox_inches='tight')

<a id='assess-thermo-chemistry-prediction'></a>

***

## Assess the thermo-chemistry prediction

#### The cells below will only work if the `t_coordinates` vector used in the PC-transport simulation is the same as the `time_grid` vector for that dataset.

[Go to the top ↑](#top-page)

In [ ]:
state_rbf_metrics = analysis.RegressionAssessment(state_space_test_trajectory[start_simulation::,selected_state_variables], state_space_ROM_predicted_rbf, variable_names=list(state_space_names[selected_state_variables]))
state_rbf_metrics.print_metrics(table_format=['pandas'], float_format='.8f', metrics=['R2', 'MSLE', 'NRMSE'])

In [ ]:
state_ann_metrics = analysis.RegressionAssessment(state_space_test_trajectory[start_simulation::,selected_state_variables], state_space_ROM_predicted_ann, variable_names=list(state_space_names[selected_state_variables]))
state_ann_metrics.print_metrics(table_format=['pandas'], float_format='.8f', metrics=['R2', 'MSLE', 'NRMSE'], comparison=state_rbf_metrics)

In [ ]:
state_kreg_metrics = analysis.RegressionAssessment(state_space_test_trajectory[start_simulation::,selected_state_variables], state_space_ROM_predicted_kreg, variable_names=list(state_space_names[selected_state_variables]))
state_kreg_metrics.print_metrics(table_format=['pandas'], float_format='.8f', metrics=['R2', 'MSLE', 'NRMSE'], comparison=state_rbf_metrics)

In [ ]:
state_gpr_metrics = analysis.RegressionAssessment(state_space_test_trajectory[start_simulation::,selected_state_variables], state_space_ROM_predicted_gpr, variable_names=list(state_space_names[selected_state_variables]))
state_gpr_metrics.print_metrics(table_format=['pandas'], float_format='.8f', metrics=['R2', 'MSLE', 'NRMSE'], comparison=state_rbf_metrics)

***